In [ ]:
# --- Cell 1: Imports and Setup ---
import sys
import os
import json
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QPushButton, QLabel,
    QFileDialog, QVBoxLayout, QWidget, QGridLayout, QScrollArea,
    QAction, QMenu, QMessageBox
)
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt, pyqtSignal

# Pillow (PIL) for image handling
from PIL import Image


In [ ]:
# --- Cell 2: ClickableLabel Class ---
class ClickableLabel(QLabel):
    clicked = pyqtSignal(str)  # emits the image path

    def __init__(self, path, parent=None):
        super().__init__(parent)
        self.path = path

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.clicked.emit(self.path)


# --- Cell 3: DRDOGUI Class Initialization ---
class DRDOGUI(QMainWindow):
    SAVE_FILE = "app_state.json"

    def __init__(self):
        super().__init__()
        self.setWindowTitle("Basic Frontend")
        self.setGeometry(200, 200, 1200, 800)

        self.current_image_path = None
        self.current_pixmap = None
        self.original_size = None
        self.thumbnail_labels = []
        self.base_metadata_text = ""
        self.current_folder = None
        self.category_history = []
        self.last_selected_path = None

        # User manual link (edit if moved)
        self.user_manual_url = "file:///C:/Users/Lenovo/Desktop/User%20Manual.pdf"

        # Toolbar setup
        self._create_toolbar()

        # Main layout
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QGridLayout(self.central_widget)

        # UI sections
        self._setup_left_panel()
        self._setup_image_display()
        self._setup_process_section()
        self._setup_category_section()
        self._setup_metadata_section()

        # Layout proportions
        self.layout.setColumnStretch(0, 1)
        self.layout.setColumnStretch(1, 2)
        self.layout.setColumnStretch(2, 2)
        self.layout.setRowStretch(0, 2)
        self.layout.setRowStretch(1, 1)

        # Load previous session
        self._load_progress()
        if self.current_folder:
            self.category_label.setText(f"Loaded {len(self.category_history)} previous categorizations.")


In [ ]:
# --- Cell 4: Toolbar Setup ---
def _create_toolbar(self):
        toolbar = self.addToolBar("Main Toolbar")

        save_action = QAction("Save", self)
        save_action.triggered.connect(self.save_progress)
        toolbar.addAction(save_action)

        undo_action = QAction("Undo", self)
        undo_action.triggered.connect(self.undo_action)
        toolbar.addAction(undo_action)

        help_action = QAction("Help", self)
        help_menu = QMenu(self)
        manual_action = QAction("User Manual", self)
        manual_action.triggered.connect(self.open_user_manual)
        help_menu.addAction(manual_action)
        help_action.setMenu(help_menu)
        toolbar.addAction(help_action)


In [ ]:
# --- Cell 5: UI Section Setup ---
def _setup_left_panel(self):
        self.upload_btn = QPushButton("Upload Folder")
        self.upload_btn.clicked.connect(self.open_folder)

        self.scroll_area = QScrollArea()
        self.scroll_area.setWidgetResizable(True)
        self.scroll_content = QWidget()
        self.scroll_layout = QVBoxLayout(self.scroll_content)
        self.scroll_layout.setAlignment(Qt.AlignTop)
        self.scroll_area.setWidget(self.scroll_content)

        vbox_left = QVBoxLayout()
        vbox_left.addWidget(self.upload_btn)
        vbox_left.addWidget(self.scroll_area)

        section_left = QWidget()
        section_left.setLayout(vbox_left)
        self.layout.addWidget(section_left, 0, 0, 2, 1)

def _setup_image_display(self):
        self.image_label = QLabel("Click a thumbnail to show here")
        self.image_label.setStyleSheet("border: 1px solid gray; min-height: 200px;")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setScaledContents(True)
        self.layout.addWidget(self.image_label, 0, 1)

def _setup_process_section(self):
        self.process_btn = QPushButton("Process (Not Implemented)")
        self.output_label = QLabel("Processed output will appear here")
        self.output_label.setStyleSheet("border: 1px solid gray; min-height: 100px;")

        vbox_proc = QVBoxLayout()
        vbox_proc.addWidget(self.process_btn)
        vbox_proc.addWidget(self.output_label)
        section_proc = QWidget()
        section_proc.setLayout(vbox_proc)
        self.layout.addWidget(section_proc, 0, 2)

def _setup_category_section(self):
        self.categorize_btn = QPushButton("Categorize (Repair / Accept)")
        self.categorize_btn.clicked.connect(self.categorize_image)
        self.category_label = QLabel("Category result will appear here")
        self.category_label.setStyleSheet("border: 1px solid gray; min-height: 50px;")

        vbox_cat = QVBoxLayout()
        vbox_cat.addWidget(self.categorize_btn)
        vbox_cat.addWidget(self.category_label)
        section_cat = QWidget()
        section_cat.setLayout(vbox_cat)
        self.layout.addWidget(section_cat, 1, 1)

def _setup_metadata_section(self):
        self.metadata_label = QLabel("Metadata will appear here")
        self.metadata_label.setStyleSheet("border: 1px solid gray; min-height: 100px;")
        self.metadata_label.setAlignment(Qt.AlignTop | Qt.AlignLeft)

        vbox_meta = QVBoxLayout()
        vbox_meta.addWidget(self.metadata_label)
        section_meta = QWidget()
        section_meta.setLayout(vbox_meta)
        self.layout.addWidget(section_meta, 1, 2)


In [ ]:
# --- Cell 6: State Management (Save/Load/Undo) ---
def _load_progress(self):
        if os.path.exists(self.SAVE_FILE):
            try:
                with open(self.SAVE_FILE, 'r') as f:
                    state = json.load(f)
                    self.category_history = state.get("history", [])
                    self.current_folder = state.get("folder", None)
                    self.last_selected_path = state.get("selected_image", None)

                if self.current_folder and os.path.isdir(self.current_folder):
                    self._load_folder_on_resume(self.current_folder, self.last_selected_path)
                    QMessageBox.information(self, "Resume Session", f"Resumed session from:\n{self.current_folder}")
            except Exception as e:
                QMessageBox.warning(self, "Load Error", str(e))

def save_progress(self):
        if not self.current_folder:
            QMessageBox.warning(self, "Save Error", "No folder uploaded.")
            return
        state = {
            "folder": self.current_folder,
            "history": self.category_history,
            "selected_image": self.current_image_path
        }
        try:
            with open(self.SAVE_FILE, 'w') as f:
                json.dump(state, f, indent=4)
            QMessageBox.information(self, "Save Success", "Progress saved.")
        except Exception as e:
            QMessageBox.critical(self, "Save Error", str(e))

def undo_action(self):
        if not self.category_history:
            self.category_label.setText("Nothing to undo.")
            self.unselect_current_image()
            return
        last_action = self.category_history.pop()
        self.category_label.setText(
            f"UNDONE: {os.path.basename(last_action['path'])} ({last_action['category']})"
        )
        self.unselect_current_image()

def unselect_current_image(self):
        self.current_image_path = None
        self.current_pixmap = None
        self.original_size = None
        self.last_selected_path = None
        self.image_label.setPixmap(QPixmap())
        self.image_label.setText("Click a thumbnail to show here")
        self.metadata_label.setText("Metadata will appear here")
        for thumb in self.thumbnail_labels:
            thumb.setStyleSheet("border: 1px solid gray; margin: 2px;")


In [ ]:
# --- Cell 7: Folder and Thumbnails ---
def open_user_manual(self):
        import webbrowser
        if self.user_manual_url:
            webbrowser.open(self.user_manual_url)
        else:
            print("User manual link not configured.")

def open_folder(self):
        folder = QFileDialog.getExistingDirectory(self, "Select Folder")
        if folder:
            self.unselect_current_image()
            self.current_folder = folder
            self._load_thumbnails(folder)

def _load_folder_on_resume(self, folder, selected_path=None):
        self.unselect_current_image()
        self._load_thumbnails(folder)
        if selected_path and os.path.exists(selected_path):
            self.on_thumbnail_clicked(selected_path)

def _load_thumbnails(self, folder):
        for i in reversed(range(self.scroll_layout.count())):
            widget = self.scroll_layout.itemAt(i).widget()
            if widget:
                widget.deleteLater()
        self.thumbnail_labels.clear()

        for fname in os.listdir(folder):
            if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                path = os.path.join(folder, fname)
                pixmap = QPixmap(path).scaledToWidth(100, Qt.SmoothTransformation)
                thumb_label = ClickableLabel(path)
                thumb_label.setPixmap(pixmap)
                thumb_label.setStyleSheet("border: 1px solid gray; margin: 2px;")
                thumb_label.setCursor(Qt.PointingHandCursor)
                thumb_label.clicked.connect(self.on_thumbnail_clicked)
                self.scroll_layout.addWidget(thumb_label)
                self.thumbnail_labels.append(thumb_label)


In [ ]:
# --- Cell 8: Image Loading & Metadata ---
def on_thumbnail_clicked(self, path):
        self.last_selected_path = path
        for thumb in self.thumbnail_labels:
            thumb.setStyleSheet("border: 1px solid gray; margin: 2px;")
        clicked_label = next((t for t in self.thumbnail_labels if t.path == path), None)
        if clicked_label:
            clicked_label.setStyleSheet("border: 2px solid blue; margin: 2px;")
        self.load_image(path)

def load_image(self, path):
        self.current_image_path = path
        self.current_pixmap = QPixmap(path)
        try:
            img = Image.open(path)
            self.original_size = img.size
        except Exception:
            self.original_size = ("N/A", "N/A")
        self.resize_image_to_label()
        self.show_metadata(path)

def resize_image_to_label(self):
        if self.current_pixmap:
            scaled_pixmap = self.current_pixmap.scaled(
                self.image_label.size(),
                Qt.KeepAspectRatio,
                Qt.SmoothTransformation
            )
            self.image_label.setPixmap(scaled_pixmap)
            self.update_metadata_display()

def resizeEvent(self, event):
        self.resize_image_to_label()
        super().resizeEvent(event)

def show_metadata(self, fname):
        try:
            img = Image.open(fname)
            img_format = img.format
            img_mode = img.mode
            img_size = f"{img.size[0]} x {img.size[1]} px"
        except Exception:
            img_format, img_mode, img_size = "Unknown", "Unknown", "N/A"

        try:
            size = os.path.getsize(fname)
            if size < 1024:
                size_str = f"{size} B"
            elif size < 1024 * 1024:
                size_str = f"{size / 1024:.2f} KB"
            else:
                size_str = f"{size / (1024 * 1024):.2f} MB"
        except Exception:
            size_str = "N/A"

        self.base_metadata_text = (
            f"File: {os.path.basename(fname)}\n"
            f"Format: {img_format}\n"
            f"Original Size: {img_size}\n"
            f"Mode: {img_mode}\n"
            f"File Size: {size_str}"
        )
        self.update_metadata_display()

def update_metadata_display(self):
        if self.current_pixmap:
            disp = self.image_label.pixmap()
            dw, dh = disp.width(), disp.height() if disp else (0, 0)
            lw, lh = self.image_label.width(), self.image_label.height()
            self.metadata_label.setText(
                f"{self.base_metadata_text}\n"
                f"Displayed Pixmap Size: {dw} x {dh}px\n"
                f"Label Area Size: {lw} x {lh}px"
            )


In [ ]:
# --- Cell 9: Categorization ---
def categorize_image(self):
        if not self.current_image_path:
            self.category_label.setText("No image selected!")
            return
        import random
        category = random.choice(["Repair", "Accept"])
        self.category_label.setText(f"Result: {category}")
        action = {"path": self.current_image_path, "category": category}
        self.category_history.append(action)


In [ ]:
# --- Cell 10: Main Entry Point ---
def main():
    app = QApplication(sys.argv)
    window = DRDOGUI()
    window.showMaximized()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()
